Regression with Panel Data
===
---
* First-Differend Estimator
* Reshape Data 
* Fixed Effects Regression
* Standard Errors for Fixed Effects Regression
---

In [1]:
insheet using "/Users/tino/Desktop/TA-Econometrics-II/datasets/0507/traffic1.csv", clear
sum


(5 vars, 51 obs)


    Variable |        Obs        Mean    Std. Dev.       Min        Max
-------------+---------------------------------------------------------
       state |          0
      open90 |         51    .4313725     .500196          0          1
      open85 |         51     .372549    .4882944          0          1
    dthrte90 |         51    2.154902    .5150976        1.3        3.6
    dthrte85 |         51         2.7    .6033241        1.9        4.4


In [3]:
describe // notice that the "storage type" of state is "string"


Contains data
  obs:            51                          
 vars:             5                          
 size:           612                          
---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
              storage   display    value
variable name   type    format     label      variable label
---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
state           str2    %9s                   
open90          byte    %8.0g                 
open85          byte    %8.0g                 
dthrte90        float   %9.0g                 
dthrte85        float   %9.0g             

In [6]:
list // wide format


     +-----------------------------------------------+
     | state   open90   open85   dthrte90   dthrte85 |
     |-----------------------------------------------|
  1. |    AL        0        0        2.6        2.9 |
  2. |    AK        1        0        2.1        3.2 |
  3. |    AZ        0        0        2.5        4.4 |
  4. |    AR        0        0        2.9        3.4 |
  5. |    CA        1        1          2        2.6 |
     |-----------------------------------------------|
  6. |    CO        0        0        1.9        2.4 |
  7. |    CT        0        0        1.5          2 |
  8. |    DE        0        0        2.2        2.2 |
  9. |    DC        0        0        1.6          3 |
 10. |    FL        1        0        2.7        3.4 |
     |-----------------------------------------------|
 11. |    GA        0        0          2        2.7 |
 12. |    HI        1        1        2.3          2 |
 13. |    ID        1        1        2.9        3.5 |
 14. |   

---
## 1. First-Differenced Estimator
* gen the "comparison variables"
* regress with comparison variables (without intercept)

In [7]:
gen copen = open90 - open85 // comparison of O.C. laws in 1990 with in 1985
gen cdthrte = dthrte90 - dthrte85 // comparison of death rate in 1990 with in 1985

In [8]:
list in 1/10


     +-------------------------------------------------------------------+
     | state   open90   open85   dthrte90   dthrte85   copen     cdthrte |
     |-------------------------------------------------------------------|
  1. |    AL        0        0        2.6        2.9       0   -.3000002 |
  2. |    AK        1        0        2.1        3.2       1        -1.1 |
  3. |    AZ        0        0        2.5        4.4       0        -1.9 |
  4. |    AR        0        0        2.9        3.4       0         -.5 |
  5. |    CA        1        1          2        2.6       0   -.5999999 |
     |-------------------------------------------------------------------|
  6. |    CO        0        0        1.9        2.4       0   -.5000001 |
  7. |    CT        0        0        1.5          2       0         -.5 |
  8. |    DE        0        0        2.2        2.2       0           0 |
  9. |    DC        0        0        1.6          3       0        -1.4 |
 10. |    FL        1   

In [9]:
reg cdthrte copen, nocon 
// regression with no intercept --> reg y x, nocon
// nocon = noconstant
// FD estimator: coefficient = -0.97


      Source |       SS           df       MS      Number of obs   =        51
-------------+----------------------------------   F(1, 50)        =      7.46
       Model |  2.80333352         1  2.80333352   Prob > F        =    0.0087
    Residual |  18.7766667        50  .375533334   R-squared       =    0.1299
-------------+----------------------------------   Adj R-squared   =    0.1125
       Total |  21.5800002        51  .423137259   Root MSE        =    .61281

------------------------------------------------------------------------------
     cdthrte |      Coef.   Std. Err.      t    P>|t|     [95% Conf. Interval]
-------------+----------------------------------------------------------------
       copen |  -.9666667   .3538047    -2.73   0.009    -1.677304    -.256029
------------------------------------------------------------------------------


---
## 2. Reshape Data
* wide format --> long format
* long format: panel regression

In [11]:
reshape long open dthrte, i(state) j(year) // --> reshape data from wide to long format
// reshape wide open dthrte, i(state) j(year) --> reshape data from long to wide format

(note: j = 85 90)

Data                               wide   ->   long
-----------------------------------------------------------------------------
Number of obs.                       51   ->     102
Number of variables                   7   ->       6
j variable (2 values)                     ->   year
xij variables:
                          open85 open90   ->   open
                      dthrte85 dthrte90   ->   dthrte
-----------------------------------------------------------------------------


In [13]:
list in 1/10 // long format


     +--------------------------------------------------+
     | state   year   open   dthrte   copen     cdthrte |
     |--------------------------------------------------|
  1. |    AK     85      0      3.2       1        -1.1 |
  2. |    AK     90      1      2.1       1        -1.1 |
  3. |    AL     85      0      2.9       0   -.3000002 |
  4. |    AL     90      0      2.6       0   -.3000002 |
  5. |    AR     85      0      3.4       0         -.5 |
     |--------------------------------------------------|
  6. |    AR     90      0      2.9       0         -.5 |
  7. |    AZ     85      0      4.4       0        -1.9 |
  8. |    AZ     90      0      2.5       0        -1.9 |
  9. |    CA     85      1      2.6       0   -.5999999 |
 10. |    CA     90      1        2       0   -.5999999 |
     +--------------------------------------------------+


---
## 3. Fixed Effects Regression
* Fixed Effects without binary varibles
    * encode x, gen(x_no)
    * xtset x t
    * xtreg y x, fe
* Fixed Effects with binary varibles
    * reg 

### Fixed Effects without Binary Varibles

In [14]:
encode state, gen(state_no)

In [20]:
list in 1/5


     +-------------------------------------------------------------+
     | state   year   open   dthrte   copen     cdthrte   state_no |
     |-------------------------------------------------------------|
  1. |    AK     85      0      3.2       1        -1.1         AK |
  2. |    AK     90      1      2.1       1        -1.1         AK |
  3. |    AL     85      0      2.9       0   -.3000002         AL |
  4. |    AL     90      0      2.6       0   -.3000002         AL |
  5. |    AR     85      0      3.4       0         -.5         AR |
     +-------------------------------------------------------------+


In [16]:
xtset state_no year

       panel variable:  state_no (strongly balanced)
        time variable:  year, 85 to 90, but with gaps
                delta:  1 unit


In [18]:
xtreg dthrte open, fe
// Fixed Effects Regression (without binary variables): coefficient = -0.97


Fixed-effects (within) regression               Number of obs     =        102
Group variable: state_no                        Number of groups  =         51

R-sq:                                           Obs per group:
     within  = 0.1299                                         min =          2
     between = 0.0015                                         avg =        2.0
     overall = 0.0061                                         max =          2

                                                F(1,50)           =       7.46
corr(u_i, Xb)  = -0.6210                        Prob > F          =     0.0087

------------------------------------------------------------------------------
      dthrte |      Coef.   Std. Err.      t    P>|t|     [95% Conf. Interval]
-------------+----------------------------------------------------------------
        open |  -.9666667   .3538047    -2.73   0.009    -1.677304    -.256029
       _cons |   2.816013   .1485467    18.96   0.000     2.5176

### Fixed Effects with Binary Variables

In [23]:
reg dthrte open i.state_no
// Fixed Effects Regression (with binary variable): coefficient = -0.97


      Source |       SS           df       MS      Number of obs   =       102
-------------+----------------------------------   F(51, 50)       =      3.10
       Model |  29.6548049        51  .581466762   Prob > F        =    0.0001
    Residual |  9.38833336        50  .187766667   R-squared       =    0.7595
-------------+----------------------------------   Adj R-squared   =    0.5143
       Total |  39.0431382       101  .386565725   Root MSE        =    .43332

------------------------------------------------------------------------------
      dthrte |      Coef.   Std. Err.      t    P>|t|     [95% Conf. Interval]
-------------+----------------------------------------------------------------
        open |  -.9666667   .3538047    -2.73   0.009    -1.677304    -.256029
             |
    state_no |
         AL  |  -.3833333   .4680396    -0.82   0.417    -1.323419     .556752
         AR  |   .0166668   .4680396     0.04   0.972    -.9234185    .9567521
         AZ  |   .31

## 4. Standard Errors for Fixed Effects Regression
* vce(cluster clusvar)
* vce(robust) or r 

In [24]:
xtreg dthrte open, fe vce(cluster state)


Fixed-effects (within) regression               Number of obs      =       102
Group variable: state_no                        Number of groups   =        51

R-sq:  within  = 0.1299                         Obs per group: min =         2
       between = 0.0015                                        avg =       2.0
       overall = 0.0061                                        max =         2

                                                F(1,50)            =     76.53
corr(u_i, Xb)  = -0.6210                        Prob > F           =    0.0000

                                 (Std. Err. adjusted for 51 clusters in state)
------------------------------------------------------------------------------
             |               Robust
      dthrte |      Coef.   Std. Err.      t    P>|t|     [95% Conf. Interval]
-------------+----------------------------------------------------------------
        open |  -.9666667   .1104979    -8.75   0.000    -1.188608   -.7447252
       _cons

In [28]:
xtreg dthrte open, fe r


Fixed-effects (within) regression               Number of obs     =        102
Group variable: state_no                        Number of groups  =         51

R-sq:                                           Obs per group:
     within  = 0.1299                                         min =          2
     between = 0.0015                                         avg =        2.0
     overall = 0.0061                                         max =          2

                                                F(1,50)           =      76.53
corr(u_i, Xb)  = -0.6210                        Prob > F          =     0.0000

                              (Std. Err. adjusted for 51 clusters in state_no)
------------------------------------------------------------------------------
             |               Robust
      dthrte |      Coef.   Std. Err.      t    P>|t|     [95% Conf. Interval]
-------------+----------------------------------------------------------------
        open |  -.9666667   